# ParallelGNN trf learning model tuning
1. This notebook will be focused on hyperparameter tuning for the trf learning model.
2. Hyperparameters were tuned using solubility dataset which is in the data/graph_data/logS_train and logS_val folders
3. To prevent any sort of test leakage, molecules that appear in both solubililty and oral bioavailability dataset have been removed from the solubililty dataset prior to tuning

## Note:
Suggest to skip this notebook and refer to the best parameters which were obtained and saved in config.py as best_params_parallel

In [ ]:
# import all required materials

import torch
import numpy as np
from torch_geometric.loader import DataLoader
import optuna

from model import VerticalGNN
from config import NUM_FEATURES, NUM_TARGET, EDGE_DIM, DEVICE, SEED_NO, PATIENCE, EPOCHS, NUM_GRAPHS_PER_BATCH
from engine import EngineSol
from utils import seed_everything, LoadSolDataset

In [6]:
seed_everything(SEED_NO)

train_dataset = LoadSolDataset(root='./data/graph_data/logS_train/', raw_filename='trainset_removed.csv')
val_dataset = LoadSolDataset(root='./data/graph_data/logS_val/', raw_filename='valset_removed.csv')
print(len(train_dataset))
print(len(val_dataset))

7880
978


In [7]:
def run_tuning(train_dataset, valid_dataset, params):
    print(len(train_dataset))
    print(len(valid_dataset))
    seed_everything(SEED_NO)
    train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

    model = VerticalGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], 
                            hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineSol(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0

        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

In [8]:
def objective(trial):
    params = {
        'num_gin_layers' : trial.suggest_int('num_gin_layers', 1,3),
        'num_graph_trans_layers' : trial.suggest_int('num_graph_trans_layers', 1,3),
        'hidden_size' : trial.suggest_int('hidden_size', 64, 512),
        'n_heads' : trial.suggest_int('n_heads', 1, 5),
        'dropout': trial.suggest_float('dropout', 0.1,0.4),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-3, 9e-3, log=True)
    }
    
    loss = run_tuning(train_dataset, val_dataset, params)
    return loss

if __name__ == '__main__':
    study = optuna.create_study(direction = 'minimize')
    study.optimize(objective, n_trials=30)
    print(f'best trial:')
    trial_ = study.best_trial
    print(trial_.values)
    best_parameters= trial_.params
    print(f'Best parameters: {trial_.params}')

[I 2023-03-10 08:53:17,112] A new study created in memory with name: no-name-bb508307-3d21-4060-9585-344b62749c62


7880
978
Epoch: 1/300, train loss : 213.725725389296, validation loss : 2.6505064964294434
Early stop counter: 0
Epoch: 2/300, train loss : 2.5098480370736893, validation loss : 1.8958339393138885
Early stop counter: 0
Epoch: 3/300, train loss : 1.9747789713644213, validation loss : 1.3982631266117096
Early stop counter: 0
Epoch: 4/300, train loss : 1.668147252451989, validation loss : 1.2364166378974915
Early stop counter: 0
Epoch: 5/300, train loss : 1.680613398551941, validation loss : 1.257998138666153
Early stop counter: 1
Epoch: 6/300, train loss : 1.5638499644494825, validation loss : 1.165939748287201
Early stop counter: 0
Epoch: 7/300, train loss : 1.4303901195526123, validation loss : 1.3134794235229492
Early stop counter: 1
Epoch: 8/300, train loss : 1.5483840011781262, validation loss : 1.1146931052207947
Early stop counter: 0
Epoch: 9/300, train loss : 1.3720417157296212, validation loss : 1.4471424520015717
Early stop counter: 1
Epoch: 10/300, train loss : 1.4321719715672

[I 2023-03-10 08:54:03,314] Trial 0 finished with value: 0.8354481160640717 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 3, 'hidden_size': 189, 'n_heads': 4, 'dropout': 0.37472528808291117, 'learning_rate': 0.0020249352160394292}. Best is trial 0 with value: 0.8354481160640717.


Epoch: 29/300, train loss : 1.8121099971955823, validation loss : 1.4559975862503052
Early stopping...
7880
978
Epoch: 1/300, train loss : 4.733111196948636, validation loss : 1.7981409430503845
Early stop counter: 0
Epoch: 2/300, train loss : 1.9210995166532454, validation loss : 1.3036916851997375
Early stop counter: 0
Epoch: 3/300, train loss : 1.7461280284389373, validation loss : 1.306392103433609
Early stop counter: 1
Epoch: 4/300, train loss : 1.6354861221005839, validation loss : 1.1859391033649445
Early stop counter: 0
Epoch: 5/300, train loss : 1.6045651820398146, validation loss : 1.281533807516098
Early stop counter: 1
Epoch: 6/300, train loss : 1.452110634696099, validation loss : 1.0378233194351196
Early stop counter: 0
Epoch: 7/300, train loss : 1.436139614351334, validation loss : 0.987675815820694
Early stop counter: 0
Epoch: 8/300, train loss : 1.3210736051682503, validation loss : 0.9302721321582794
Early stop counter: 0
Epoch: 9/300, train loss : 1.3551463965446717,

[I 2023-03-10 08:55:13,426] Trial 1 finished with value: 0.9137213826179504 and parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 3, 'hidden_size': 288, 'n_heads': 3, 'dropout': 0.36747182114767285, 'learning_rate': 0.0015090766911011915}. Best is trial 0 with value: 0.8354481160640717.


...

Epoch: 32/300, train loss : 2.08741303413145, validation loss : 1.6471209526062012
Early stopping...
7880
978
Epoch: 1/300, train loss : 78.0947995185852, validation loss : 3.243434727191925
Early stop counter: 0
Epoch: 2/300, train loss : 2.5817728350239415, validation loss : 2.107990562915802
Early stop counter: 0
Epoch: 3/300, train loss : 1.9874564947620514, validation loss : 1.4740507304668427
Early stop counter: 0
Epoch: 4/300, train loss : 1.748931307946482, validation loss : 1.4534550905227661
Early stop counter: 0
Epoch: 5/300, train loss : 1.5379351377487183, validation loss : 1.435213416814804
Early stop counter: 0
Epoch: 6/300, train loss : 1.6465086936950684, validation loss : 1.295859694480896
Early stop counter: 0
Epoch: 7/300, train loss : 1.501011436985385, validation loss : 1.2455772459506989
Early stop counter: 0
Epoch: 8/300, train loss : 1.3097092067041705, validation loss : 1.073820412158966
Early stop counter: 0
Epoch: 9/300, train loss : 1.2183992651201063, vali

[I 2023-03-10 09:14:56,917] Trial 28 finished with value: 0.6105415672063828 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 2, 'hidden_size': 325, 'n_heads': 1, 'dropout': 0.13110110483891943, 'learning_rate': 0.002814856900150019}. Best is trial 22 with value: 0.5250212922692299.


Epoch: 69/300, train loss : 0.6522394447557388, validation loss : 0.6648846119642258
Early stopping...
7880
978
Epoch: 1/300, train loss : 4.840147518342541, validation loss : 2.965992569923401
Early stop counter: 0
Epoch: 2/300, train loss : 2.7134640832101145, validation loss : 1.746374487876892
Early stop counter: 0
Epoch: 3/300, train loss : 1.8126430088473904, validation loss : 1.356557846069336
Early stop counter: 0
Epoch: 4/300, train loss : 1.5553345180326892, validation loss : 1.231900542974472
Early stop counter: 0
Epoch: 5/300, train loss : 1.4403408592747104, validation loss : 1.1192646920681
Early stop counter: 0
Epoch: 6/300, train loss : 1.3201916871532318, validation loss : 0.9840782135725021
Early stop counter: 0
Epoch: 7/300, train loss : 1.303233273567692, validation loss : 1.0536766350269318
Early stop counter: 1
Epoch: 8/300, train loss : 1.1883503833124716, validation loss : 1.004202425479889
Early stop counter: 2
Epoch: 9/300, train loss : 1.1592603268161896, val

[I 2023-03-10 09:15:26,509] Trial 29 finished with value: 0.6536026149988174 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 3, 'hidden_size': 180, 'n_heads': 1, 'dropout': 0.19130663047416607, 'learning_rate': 0.00219903817376424}. Best is trial 22 with value: 0.5250212922692299.


Epoch: 47/300, train loss : 0.7609452701384022, validation loss : 0.7134182751178741
Early stopping...
best trial:
[0.5250212922692299]
Best parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 245, 'n_heads': 1, 'dropout': 0.30146027310173296, 'learning_rate': 0.0012649520485726895}


Hyperparameters are saved in config.py as best_params_vertical